In [257]:
import numpy as np
import pandas as pd
from Levenshtein import distance as levenshtein_distance
from collections import Counter
from keras.utils.data_utils import pad_sequences
from sklearn.metrics.pairwise import cosine_similarity
from pyjarowinkler import distance as jaro_winkler_distance
import re
import string as s
import random
import torch
from transformers import AutoTokenizer, AutoModel
from gensim.models import FastText
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import openpyxl

In [258]:
def GetDisparity(price1, price2, rate, weight1=1, weight2=1):
    '''
    price1 - цена товара в первой валюте\n
    price2 - цена товара во второй валюте\n
    rate - курс валюты\n
    weight1 - вес товара в первой корзине\n
    weight2 - вес товара во второй корзине
    '''

    return (price1 * weight1 / rate) / (price2 * weight2)

# определяем функцию для получения части речи слова
def get_pos(words):
    # получаем кортеж (слово, часть речи)
    tags = nltk.pos_tag(words, lang='rus')
    for tag in tags:
        # преобразуем часть речи в удобный формат
        if tag[1].lower() == 's':
            yield (tag[0], 'noun')
        elif tag[1].lower() == 'v':
            yield (tag[0], 'noun')
        elif tag[1].lower() == 'a=f':
            yield (tag[0], 'adj')
        elif tag[1].lower() == 'a=n':
            yield (tag[0], 'adj')
        elif tag[1].lower() == 'r':
            yield (tag[0], 'adv')
        else:
            yield (tag[0], None)

# определяем функцию для получения веса слова в зависимости от его части речи
def get_weight(pos):
    # задаем словарь весов для разных частей речи
    weights = {'noun': 10, 'adj': 7, 'verb': 5, 'adv': 2, None: 0}
    # если часть речи есть в словаре, возвращаем соответствующий вес
    if pos in weights:
        return weights[pos]
    # иначе возвращаем 1
    else:
        return 1

# определяем функцию для получения списка слов с весами из строки
def get_words_with_weights(string):
    # разбиваем строку на слова
    words = nltk.word_tokenize(string)
    # создаем пустой список для хранения слов с весами
    words_with_weights = []
    poss = get_pos(words)
    # для каждого слова в строке
    for pos in poss:
        # получаем его вес
        weight = get_weight(pos[1])
        # добавляем слово с весом в список
        words_with_weights.append((pos[0], weight))
    # возвращаем список слов с весами
    return words_with_weights

# определяем функцию для получения косинусного сходства между двумя строками по новому алгоритму
def get_cosine_similarity_new(string1, string2):
    # получаем списки слов с весами из строк
    words_with_weights1 = get_words_with_weights(string1)
    words_with_weights2 = get_words_with_weights(string2)
    
    # создаем пустые списки для хранения слов и весов отдельно
    words1 = []
    weights1 = []
    words2 = []
    weights2 = []
    
    # для каждого слова с весом из первого списка
    for word, weight in words_with_weights1:
        # добавляем слово в список слов первой строки
        words1.append(word)
        # добавляем вес в список весов первой строки
        weights1.append(weight)
        
     # аналогично для каждого слова с весом из второго списка   
    for word, weight in words_with_weights2:
        words2.append(word)
        weights2.append(weight)
        
    # создаем объект CountVectorizer для преобразования слов в числовые признаки
    # устанавливаем параметр lowercase равным False, чтобы сохранить регистр слов
    vectorizer = CountVectorizer()
    
    # объединяем списки слов из обеих строк в один список
    words = words1 + words2
    
    # преобразуем список слов в матрицу признаков X
    X = vectorizer.fit_transform(words)
    
    # получаем список имен признаков (уникальных слов) из объекта vectorizer
    features = vectorizer.get_feature_names_out()
    
    # создаем пустые списки для хранения признаков с весами для обеих строк
    features_with_weights1 = []
    features_with_weights2 = []
    
    # для каждого признака (слова) в списке признаков
    for feature in features:
        # если признак есть в списке слов первой строки
        if feature in words1:
            # получаем индекс признака в списке слов первой строки
            index = words1.index(feature)
            # получаем вес признака из списка весов первой строки по индексу
            weight = weights1[index]
        # иначе
        else:
            # задаем вес признака равным 0
            weight = 0
        # добавляем признак с весом в список признаков с весами для первой строки
        features_with_weights1.append((feature, weight))
        
        # аналогично для второй строки
        if feature in words2:
            index = words2.index(feature)
            weight = weights2[index]
        else:
            weight = 0
        features_with_weights2.append((feature, weight))
        
    # создаем пустые списки для хранения сопоставленных частей между строками и их суммарных весов
    matched_parts = []
    matched_weights = []
    
    # для каждого признака с весом из первого списка
    for feature, weight in features_with_weights1:
        # если признак не равен 0 (то есть присутствует в первой строке)
        if weight != 0:
            # если признак есть во втором списке и его вес не равен 0 (то есть присутствует во второй строке)
            if feature in dict(features_with_weights2) and dict(features_with_weights2)[feature] != 0:
                # добавляем признак в список сопоставленных частей между строками
                matched_parts.append(feature)
                # добавляем сумму весов признака из обеих строк в список сопоставленных весов
                matched_weight = weight + dict(features_with_weights2)[feature]
                matched_weights.append(matched_weight)
                
    # вычисляем общий результат косинусного сходства по сопоставленным частям и их весам
    cosine_similarity_result = sum(matched_weights)
    
    # вычисляем максимальный результат косинусного сходства, который может быть достигнут при полном совпадении строк
    cosine_similarity_max = sum(weights1) + sum(weights2)
    
    if cosine_similarity_result == 0 or cosine_similarity_max == 0:
        return 0
    # вычисляем соотношение между результатом и максимумом косинусного сходства
    cosine_similarity_ratio = cosine_similarity_result / cosine_similarity_max
    
    # округляем соотношение до двух знаков после запятой
    cosine_similarity_ratio = round(cosine_similarity_ratio, 2)
    
    # возвращаем соотношение как результат функции
    return cosine_similarity_ratio

In [227]:
# Определяем функцию для вычисления косинусного сходства между двумя фразами
def my_cosine_similarity(phrase1, phrase2):
    # Токенизируем фразы и добавляем специальные токены [CLS] и [SEP]
    tokens1 = tokenizer.encode(phrase1, add_special_tokens=True)
    tokens2 = tokenizer.encode(phrase2, add_special_tokens=True)
    # Конвертируем токены в тензоры PyTorch
    tokens1 = torch.tensor([tokens1])
    tokens2 = torch.tensor([tokens2])
    # Получаем выходы модели BERT для каждого токена
    outputs1 = model(tokens1)
    outputs2 = model(tokens2)
    # Извлекаем вектора признаков для специального токена [CLS], который представляет собой эмбеддинг всей фразы
    features1 = outputs1[0][:,0,:]
    features2 = outputs2[0][:,0,:]
    # Вычисляем косинусное сходство между векторами признаков и возвращаем его
    return torch.cosine_similarity(features1, features2).item()

# Определяем функцию для сопоставления двух товаров по названию, стоимости, группе и подгруппе
def match_products(product1, product2, price1, price2, group1, group2, subgroup1, subgroup2):
    # Проверяем, есть ли None среди аргументов
    if product1 is None:
        product1 = "" # присваиваем пустую строку для названия товара
    if product2 is None:
        product2 = "" # присваиваем пустую строку для названия товара
    if price1 is None:
        price1 = 0 # присваиваем нулевую стоимость для цены товара
    if price2 is None:
        price2 = 0 # присваиваем нулевую стоимость для цены товара
    if group1 is None:
        group1 = "" # присваиваем пустую строку для группы товара
    if group2 is None:
        group2 = "" # присваиваем пустую строку для группы товара
    if subgroup1 is None:
        subgroup1 = "" # присваиваем пустую строку для подгруппы товара
    if subgroup2 is None:
        subgroup2 = "" # присваиваем пустую строку для подгруппы товара
    
    # Вычисляем косинусное сходство между названиями товаров
    similarity_name = my_cosine_similarity(product1, product2)
    # Проверяем, не равны ли обе цены нулю
    # if price1 == 0 and price2 == 0:
    #     difference_price = 0 # присваиваем нулевое значение для разницы по стоимости
    # else:
    #     # Вычисляем относительную разницу между стоимостями товаров
    #     difference_price = abs(price1 - price2) / max(price1, price2)
    # Вычисляем косинусное сходство между группами товаров
    # similarity_group = my_cosine_similarity(group1, group2)
    # Вычисляем косинусное сходство между подгруппами товаров
    # similarity_subgroup = my_cosine_similarity(subgroup1, subgroup2)
    # Взвешиваем сходство по названию, разницу по стоимости, сходство по группе и сходство по подгруппе с коэффициентами alpha, beta, gamma и delta
    alpha = 0.6 # коэффициент для сходства по названию
    beta = 0.25 # коэффициент для разницы по стоимости
    gamma = 0.1 # коэффициент для сходства по группе
    delta = 0.05 # коэффициент для сходства по подгруппе
    # Возвращаем степень сходства товаров в диапазоне от 0 до 1
    # return alpha * similarity_name - beta * difference_price + gamma * similarity_group + delta * similarity_subgroup
    return similarity_name

# Определяем функцию для вычисления частоты слов в списке фраз
def word_frequency(phrases):
  # Создаем пустой словарь для хранения частот слов
  freq = {}
  # Для каждой фразы в списке
  for phrase in phrases:
    # Разбиваем фразу на слова по пробелам
    words = phrase.split()
    # Для каждого слова в фразе
    for word in words:
      # Приводим слово к нижнему регистру
      word = word.lower()
      # Если слово уже есть в словаре, то увеличиваем его частоту на единицу
      if word in freq:
        freq[word] += 1
      # Иначе добавляем слово в словарь с частотой единица
      else:
        freq[word] = 1
  # Возвращаем словарь частот слов
  return freq

# Определяем функцию для вычисления взвешенного косинусного сходства между двумя фразами с учетом частоты слов в списках фраз
def weighted_cosine_similarity(phrase1, phrase2, phrases1, phrases2):
  # Вычисляем косинусное сходство между фразами с использованием Sentence-BERT
  similarity = my_cosine_similarity(phrase1, phrase2)
  # Вычисляем частоту слов в первом списке фраз
  freq1 = word_frequency(phrases1)
  # Вычисляем частоту слов во втором списке фраз
  freq2 = word_frequency(phrases2)
  # Вычисляем сумму частот слов в обоих списках фраз
  total_freq = sum(freq1.values()) + sum(freq2.values())
  # Вычисляем среднюю частоту слов в обоих списках фраз
  mean_freq = total_freq / (len(freq1) + len(freq2))
  # Разбиваем первую фразу на слова по пробелам
  words1 = phrase1.split()
  # Разбиваем вторую фразу на слова по пробелам
  words2 = phrase2.split()
  # Создаем пустой список для хранения весов слов
  weights = []
  # Для каждого слова в первой фразе
  for word in words1:
    # Приводим слово к нижнему регистру
    word = word.lower()
    # Если слово есть в словаре частот первого списка фраз, то берем его частоту
    if word in freq1:
      weight = freq1[word]
    # Иначе берем среднюю частоту слов
    else:
      weight = mean_freq
    # Добавляем вес слова в список весов
    weights.append(weight)
  # Для каждого слова во второй фразе
  for word in words2:
    # Приводим слово к нижнему регистру
    word = word.lower()
    # Если слово есть в словаре частот второго списка фраз, то берем его частоту
    if word in freq2:
      weight = freq2[word]
    # Иначе берем среднюю частоту слов
    else:
      weight = mean_freq
    # Добавляем вес слова в список весов
    weights.append(weight)
  # Вычисляем средний вес слов в обоих фразах
  mean_weight = sum(weights) / len(weights)
  # Вычисляем коэффициент для усиления или ослабления косинусного сходства в зависимости от среднего веса слов
  coeff = mean_weight / mean_freq
  # Возвращаем взвешенное косинусное сходство между фразами, умноженное на коэффициент
  return similarity * coeff

# Определяем функцию для запрашивания двух списков групп, затем сопоставления названий групп, учитывая частоту слов в этих списках
def match_groups(groups1, groups2):
  results = []
  # Для каждой группы из первого списка
  for group1 in groups1:
    # Создаем пустой словарь для хранения сопоставленных групп и их сходства из второго списка
    matches = {}
    # Для каждой группы из второго списка
    for group2 in groups2:
      # Вычисляем взвешенное косинусное сходство между группами с учетом частоты слов в списках групп
      similarity = weighted_cosine_similarity(group1, group2, groups1, groups2)
      # Добавляем группу из второго списка и ее сходство с группой из первого списка в словарь сопоставленных групп
      matches[group2] = similarity
    # Сортируем словарь сопоставленных групп по убыванию сходства
    matches = sorted(matches.items(), key=lambda x: x[1], reverse=True)
    # Берем первую группу из отсортированного словаря, как наиболее похожую на группу из первого списка
    best_match = matches[0][0]
    # Берем вторую группу из отсортированного словаря, как вторую по похожести на группу из первого списка
    second_match = matches[1][0]
    # Добавляем группу из первого списка, ее наиболее похожую и вторую по похожести группы из второго списка и их сходства в список результатов сопоставления групп
    results.append((group1, best_match, second_match, matches[0][1], matches[1][1]))
  # Выводим список результатов сопоставления групп в виде таблицы
  print("Результаты сопоставления групп:")
  print("| Группа из первого списка | Наиболее похожая группа из второго списка | Вторая по похожести группа из второго списка | Сходство с наиболее похожей группой | Сходство со второй по похожести группой |")
  print("|--------------------------|-------------------------------------------|--------------------------------------------|------------------------------------|-------------------------------------|")
  for result in results:
    print(f"| {result[0]} | {result[1]} | {result[2]} | {result[3]:.2f} | {result[4]:.2f} |")

def ngrams(word, n):
  return [word[i:i+n] for i in range(len(word)-n+1)]

# Функция для сравнения двух строк с помощью косинусного расстояния и расстояния Левенштейна-Яровского
def compare_cosine_lev_jar(str1, str2, n):
  # Преобразование строк в наборы n-грамм
  set1 = set(ngrams(str1, n))
  set2 = set(ngrams(str2, n))
  # Вычисление косинусного расстояния между наборами
  cosine_dist = 1 - len(set1.intersection(set2)) / (len(set1) * len(set2))**0.5
  # Вычисление расстояния Левенштейна-Яровского между строками
  lev_jar_dist = nltk.jaccard_distance(set(nltk.ngrams(str1, n)), set(nltk.ngrams(str2, n)))
  # Возвращение кортежа из двух значений
  return (cosine_dist, lev_jar_dist)

# Функция для сопоставления двух строк с помощью всех комбинаций n-грамм длиной от 3 до 6 символов и возвращения самого высокого значения сходства
def compare_all_ngrams_and_return_max_similarity(str1, str2):
  # Создание пустого списка для хранения результатов
  results = []
  # Перебор всех возможных значений n от 3 до 6
  for n in range(3,7):
    # Вызов функции compare_cosine_lev_jar для текущего значения n и добавление результата в список
    results.append(compare_cosine_lev_jar(str1, str2, n))
  # Вычисление максимального значения сходства как минимального значения косинусного расстояния или расстояния Левенштейна-Яровского из списка результатов
  max_similarity = min(min(results))
  # Возвращение максимального значения сходства
  return max_similarity

Загрузить все товары РБ из файла с ценами

In [259]:
df = pd.read_excel("Average_prices-06-2023.xls", header=6)
df = df.rename(columns={df.columns[0]: 'Название товара'})

Выделить только продовольственные товары и очистить от мусора

In [260]:
prodPricesStartIndex = df[df['Название товара'].str.lower() == 'продовольственные товары'].index[0]
prodPricesEndIndex = df[df['Название товара'].str.lower() == 'непродовольственные товары'].index[0]
dfProdPrices = df[prodPricesStartIndex+1:prodPricesEndIndex]
# Удаляет все строки, где хоть в одной ячейке есть nan. Заменить на удаление только тех строк, где вообще все ячейки пустые
dfProdPrices = dfProdPrices.dropna()

Выделить только непродовольственные товары и очистить от мусора

In [261]:
noProdPricesStartIndex = df[df['Название товара'].str.lower() == 'непродовольственные товары'].index[0]
dfNoProdPrices = df[noProdPricesStartIndex+1:]
# Удаляет все строки, где хоть в одной ячейке есть nan. Заменить на удаление только тех строк, где вообще все ячейки пустые
dfNoProdPrices = dfNoProdPrices.dropna()

Загрузка всех товаров РБ из файла с весами

In [262]:
df2 = pd.read_excel("! Копия Веса.xlsx", sheet_name='РБ_Струкутура', header=8)
dfProdWeights = df2.iloc[:df2[df2['ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ (без табачных изделий)'] == 'НЕПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'].index[0]]
df2 = pd.read_excel("! Копия Веса.xlsx", sheet_name='РБ_Струкутура', header=8)
# df = df.rename(columns={'ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ (без табачных изделий)':'name'})
dfNoProdWeights = df2.iloc[df2[df2['ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ (без табачных изделий)'] == 'НЕПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'].index[0]+1:]


In [263]:
# Класс Product, который имеет свойства Имя, Цены, Доля, Родительский класс
class Product:
    def __init__(self, name):
        self.name = name
        self.weight = None
        self.pricesBYN = dict()
        self.pricesRUB = dict()
        self.subgroup = None
        self.group = None
    
    def SetGroup(self, group):
        self.group = group

    def GetGroup(self):
        return self.group

    def SetSubgroup(self, subgroup):
        self.subgroup = subgroup
    
    def GetSubgroup(self):
        return self.subgroup

    def SetWeight(self, weight):
        self.weight = weight
    
    def SetPriceBYN(self, state, price):
        self.pricesBYN[state] = price

    def SetPriceRUB(self, state, price):
        self.pricesRUB[state] = price

# Класс Subgroup, который имеет Имя, список объектов Product, метод, который возвращает список объектов Product
class Subgroup:
    def __init__(self, name):
        self.name = name
        self.products = [] # список объектов Product
        self.group = None

    # Метод, который возвращает список объектов Product
    def GetProducts(self):
        return self.products
    
    def GetGroup(self):
        return self.group
    
    def SetGroup(self, group):
        self.group = group

    # Функция Append, которая добавляет объект Product в список products
    def Append(self, product):
        if isinstance(product, Product): # проверяем, что аргумент является объектом Product
            self.products.append(product) # добавляем его в список products
        else:
            print("Invalid argument") # иначе выводим сообщение об ошибке

# Класс Group, который имеет Имя, список объектов Product, список объектов Subgroup,
# имеет метод, который возвращает список объектов Product, который содержит сам и который содержится в объектах Subgroup
class Group:
    def __init__(self, name):
        self.name = name
        self.products = [] # список объектов Product
        self.subgroups = [] # список объектов Subgroup

    # Метод, который возвращает список объектов Product, который содержит сам и который содержится в объектах Subgroup
    def GetProducts(self):
        result = [] # пустой список для результата
        result.extend(self.products) # добавляем в него свои продукты
        for subgroup in self.subgroups: # проходим по подгруппам
            result.extend(subgroup.GetProducts()) # добавляем в него продукты из подгрупп
        return result
    
    def GetSubgroups(self):
        return self.subgroups

    # Функция Append, которая добавляет объект Product или Subgroup в соответствующий список
    def Append(self, item):
        if isinstance(item, Product): # проверяем, что аргумент является объектом Product
            self.products.append(item) # добавляем его в список products
        elif isinstance(item, Subgroup): # проверяем, что аргумент является объектом Subgroup
            self.subgroups.append(item) # добавляем его в список subgroups
        else:
            print("Invalid argument") # иначе выводим сообщение об ошибке

# Класс Category, который имеет Имя и метод получения списка всех объектов Product у всех дочерних объектов
class Category:
    def __init__(self, name):
        self.name = name
        self.groups = [] # список объектов Group

    # Метод получения списка всех объектов Product у всех дочерних объектов
    def GetProducts(self):
        result = [] # пустой список для результата
        for group in self.groups: # проходим по группам
            result.extend(group.GetProducts()) # добавляем в него продукты из групп и подгрупп
        return result
    
    def GetGroups(self):
        return self.groups

    # Функция Append, которая добавляет объект Group в список groups
    def Append(self, group):
        if isinstance(group, Group): # проверяем, что аргумент является объектом Group
            self.groups.append(group) # добавляем его в список groups
        else:
            print("Invalid argument") # иначе выводим сообщение об ошибке
            
    def to_excel(self, filename, exchange_rate):
        # exchange_rate is the currency conversion rate from BYN to RUB
        # filename is the name of the Excel file to write to
        wb = openpyxl.Workbook() # create a new workbook
        ws = wb.active # get the active worksheet
        columns = ["Название", 'Доля РБ'] # list of column names for the Excel table
        regions_RUB = set() # set of regions in RUB
        regions_BYN = set() # set of regions in BYN
        for group in self.groups:
            for subgroup in group.subgroups:
                for product in subgroup.products:
                    for region in product.pricesRUB.keys():
                        regions_RUB.add(region) # add region to the set
                    for region in product.pricesBYN.keys():
                        regions_BYN.add(region) # add region to the set
        
        regions_RUB = sorted(regions_RUB) # sort regions alphabetically
        regions_BYN = sorted(regions_BYN) # sort regions alphabetically
        
        for i in range(len(regions_RUB)):
            for j in range(len(regions_BYN)):
                columns.append(f"{regions_BYN[j]}-{regions_RUB[i]}") # add column name for disparity
        
        ws.append(columns) # write column names to the first row
        
        row_index = 2 # start from the second row
        
        for group in self.groups:
            ws.cell(row=row_index, column=1).value = f"{group.name}" # write group name to the first column with a slash at the end
            ws.cell(row=row_index, column=1).font = openpyxl.styles.Font(bold=True) # make group name bold
            row_index += 1 # increment row index
            for product in group.products: # add products that are in the group list but not in the subgroup list
                ws.cell(row=row_index, column=1).value = f"{product.name}" # write group name and product name to the first column separated by a slash
                if product.weight != None:
                        ws.cell(row=row_index, column=2).value = product.weight
                for i in range(len(regions_RUB)):
                    for j in range(len(regions_BYN)):
                        price_RUB = product.pricesRUB.get(regions_RUB[i], None) # get the price in RUB for the region, or None if not available
                        price_BYN = product.pricesBYN.get(regions_BYN[j], None) # get the price in BYN for the region, or None if not available
                        if price_RUB and price_BYN: # if both prices are available
                            # disparity = price_BYN * exchange_rate / price_RUB # calculate disparity using exchange rate and prices
                            disparity = GetDisparity(price_BYN, price_RUB, exchange_rate)
                            ws.cell(row=row_index, column=3 + i * len(regions_BYN) + j).value = disparity # write disparity to the corresponding column
                        else: # if one or both prices are not available
                            ws.cell(row=row_index, column=3 + i * len(regions_BYN) + j).value = "" # write "Н/Д" (not available) to the corresponding column
                row_index += 1 # increment row index
            for subgroup in group.subgroups:
                ws.cell(row=row_index, column=1).value = f"{subgroup.name}" # write group name and subgroup name to the first column separated by a slash
                ws.cell(row=row_index, column=1).font = openpyxl.styles.Font(italic=True) # make subgroup name italic
                row_index += 1 # increment row index
                for product in subgroup.products:
                    ws.cell(row=row_index, column=1).value = f"{product.name}" # write group name, subgroup name and product name to the first column separated by slashes
                    if product.weight != None:
                        ws.cell(row=row_index, column=2).value = product.weight
                    for i in range(len(regions_RUB)):
                        for j in range(len(regions_BYN)):
                            price_RUB = product.pricesRUB.get(regions_RUB[i], None) # get the price in RUB for the region, or None if not available
                            price_BYN = product.pricesBYN.get(regions_BYN[j], None) # get the price in BYN for the region, or None if not available
                            if price_RUB and price_BYN: # if both prices are available
                                # disparity = price_BYN * exchange_rate / price_RUB # calculate disparity using exchange rate and prices
                                disparity = GetDisparity(price_BYN, price_RUB, exchange_rate)
                                ws.cell(row=row_index, column=3 + i * len(regions_BYN) + j).value = disparity # write disparity to the corresponding column
                            else: # if one or both prices are not available
                                ws.cell(row=row_index, column=3 + i * len(regions_BYN) + j).value = "" # write "Н/Д" (not available) to the corresponding column
                    row_index += 1 # increment row index
        wb.save(filename) # save workbook to Excel file


In [264]:
wb = openpyxl.load_workbook("! Копия Веса.xlsx")
ws = wb.get_sheet_by_name('РБ_Струкутура')

def FindIndexByText(ws, column, text):
    for i in range(9, ws.max_row + 1):
        # Получаем значение ячейки в первом столбце
        value = ws.cell(i, column).value
        # Сравниваем значение с искомым текстом
        if value != None and text.lower() in value.lower():
            # Выводим индекс строки
            return i
        
prodIndex = FindIndexByText(ws, 1, 'Продовольственные товары')
noProdIndex = FindIndexByText(ws, 1, 'Непродовольственные товары')
serviceIndex = FindIndexByText(ws, 1, 'Услуги')

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\4057060405.py:2: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  ws = wb.get_sheet_by_name('РБ_Струкутура')


In [265]:
prodCategoryRB = Category('Продовольственные товары')

lastGroup = None
lastSubgroup = None

for i in range(prodIndex+1, noProdIndex):
    cell = ws.cell(i, 1)
    weightCell = ws.cell(i,2)
    value = cell.value
    weight = weightCell.value
    font = cell.font
    if weight:
        if font.bold and not font.italic and not font.underline:
            print('Group: ',value)
            group = Group(value)
            prodCategoryRB.Append(group)
            lastGroup = group
            lastSubgroup = None
        elif not font.bold and font.italic and font.underline:
            print('Subgroup', value)
            subgroup = Subgroup(value)
            group.Append(subgroup)
            lastSubgroup = subgroup
        elif not font.bold and not font.italic and not font.underline:
            print('Product:', value, '-', weight)
            product = Product(value)
            product.SetWeight(weight)
            if lastSubgroup != None:
                product.SetSubgroup(lastSubgroup)
                product.SetGroup(lastGroup)
                lastSubgroup.Append(product)
            else:
                product.SetGroup(lastGroup)
                lastGroup.Append(product)
        

Group:  МЯСО И МЯСОПРОДУКТЫ
Product: говядина (кроме бескостного мяса) - 0.0011
Product: говядина бескостная - 0.0001
Product: свинина (кроме бескостного мяса) - 0.0151
Product: свинина бескостная - 0.0018
Product: фарш мясной  - 0.0114
Product: рагу, набор из свинины - 0.0016
Product: субпродукты мясные пищевые (кроме субпродуктов из птицы) - 0.0011
Product: куры (цыплята, включая бройлеров) - 0.0042
Product: полуфабрикаты из мяса птицы - 0.0137
Product: субпродукты из птицы - 0.0009
Product: котлеты мясные - 0.0007
Product: пельмени мясные - 0.0041
Product: сало, грудинка - 0.0036
Product: готовые изделия (вареные, жареные, фаршированные) из теста с мясной начинкой  - 0.0019
Product: готовые изделия (вареные, жареные, фаршированные, заливные) из мяса и субпродуктов  - 0.001
Subgroup Колбасные изделия
Product: колбаса вареная высшего сорта - 0.0079
Product: колбаса вареная первого сорта, второго сорта и бессортовая - 0.0005
Product: колбаса полукопченая, варено-копченая - 0.0065
Produ

Непродовольственные товары

In [266]:
noProdCategoryRB = Category('Непродовольственные товары')


lastGroup = None
lastSubgroup = None

for i in range(noProdIndex+1, serviceIndex):
    cell = ws.cell(i, 1)
    value = cell.value
    weightCell = ws.cell(i, 2)
    weight = weightCell.value
    font = cell.font
    
    if weight:
        if font.bold and not font.italic and not font.underline:
            print('Group: ',value)
            group = Group(value)
            noProdCategoryRB.Append(group)
            lastGroup = group
            lastSubgroup = None
        elif font.bold and font.italic:
            print('Subgroup', value)
            subgroup = Subgroup(value)
            group.Append(subgroup)
            lastSubgroup = subgroup
        elif not font.bold and not font.italic and not font.underline:
            print('Product:', value, '-', weight)
            product = Product(value)
            product.SetWeight(weight)
            if lastSubgroup != None:
                product.SetSubgroup(lastSubgroup)
                product.SetGroup(lastGroup)
                lastSubgroup.Append(product)
            else:
                product.SetGroup(lastGroup)
                lastGroup.Append(product)

Group:  ТКАНИ И ПРЯЖА
Product: хлопчатобумажные - 0.0003
Product: шерстяные и полушерстяные - 0.0001
Product: льняные - 0.0001
Product: шелковые - 0.0001
Product: пряжа шерстяная (полушерстяная), смесовая - 0.0002
Group:  ОДЕЖДА
Subgroup мужская
Product: куртка утепленная  - 0.002
Product: пальто (полупальто) демисезонное  - 0.0001
Product: куртка (типа ветровки) - 0.0002
Product: костюм - 0.0004
Product: джинсы - 0.0009
Product: брюки - 0.0009
Product: сорочка верхняя  - 0.0007
Product: костюм спортивный  - 0.001
Product: свитер, джемпер из шерстяной или полушерстяной пряжи - 0.0003
Product: свитер, джемпер из синтетической или смесовой пряжи - 0.0007
Product: майка - 0.0002
Product: трусы - 0.0007
Product: фуфайка, тенниска из хлопчатобумажного или смесового трикотажного полотна - 0.0009
Product: носки из хлопчатобумажной и смесовой пряжи - 0.0007
Product: носки из полушерстяной пряжи - 0.0002
Product: шляпа, кепка, бейсболка - 0.0001
Product: пиджак из полушерстяной, синтетической и

Услуги

In [236]:
serviceCategory = Category('Услуги')

for i in range(serviceIndex+1, ws.max_row + 1):
    cell = ws.cell(i, 1)
    weightCell = ws.cell(i, 2)
    if cell.value and weightCell.value:
        value = cell.value
        font = cell.font
        alignment = cell.alignment
        indent = alignment.indent
        # print('Font:', font.name, 'Indent:', indent, '-', value)
        if font.bold and not font.italic and not font.underline:
            print('Group: ',value)
            group = Group(value)
            serviceCategory.Append(group)
            lastList = group
        # elif font.bold and font.italic:
        #     print('SubGroup', value)
        #     subgroup = Subgroup(value)
        #     group.Append(subgroup)
        #     lastList = subgroup
        # elif not font.bold and not font.italic and not font.underline:
        #     print('Product:', value)
        #     product = Product(value)
        #     lastList.Append(product)

Group:  ЖИЛИЩНО-КОММУНАЛЬНЫЕ
Group:  ГОСТИНИЦ И АНАЛОГИЧНЫХ СРЕДСТВ РАЗМЕЩЕНИЯ
Group:  БЫТОВЫЕ
Group:  СВЯЗИ
Group:  ДОШКОЛЬНЫХ УЧРЕЖДЕНИЙ
Group:  ТРАНСПОРТА
Group:  ПАССАЖИРСКОГО ТРАНСПОРТА
Group:  ТРАНСПОРТНОЙ ЭКСПЕДИЦИИ
Group:  КУЛЬТУРЫ
Group:  САНАТОРНО-ОЗДОРОВИТЕЛЬНЫЕ
Group:  ТУРИСТИЧЕСКИЕ 
Group:  ФИЗИЧЕСКОЙ КУЛЬТУРЫ И СПОРТА
Group:  МЕДИЦИНСКИЕ
Group:  БАНКОВ
Group:  ПРАВОВЫЕ
Group:  ОБРАЗОВАНИЯ
Group:  УСЛУГИ ПО СДАЧЕ  В  АРЕНДУ (НАЕМ) СОБСТВЕННОГО  ИЛИ  АРЕНДОВАННОГО ЖИЛЬЯ
Group:  ПРОЧИЕ УСЛУГИ


Получение списка товаров РБ с ценами

In [267]:
def loadProdPricesRB(path='Average_prices-06-2023.xls', header=7):
    ''' Загрузить продовольственные товары РБ из файла с ценами
    '''
    df = pd.read_excel(path, header=header)
    prod = df.iloc[:df[df['ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'] == 'НЕПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'].index[0]]
    prod = prod.dropna()
    prod = prod.rename(columns={prod.columns[0]:'Название товара', prod.columns[1]:'РБ', prod.columns[2]:'Брестская', prod.columns[3]:'Витебская', prod.columns[4]:'Гомельская', prod.columns[5]:'Гродненская', prod.columns[6]:'Минск', prod.columns[7]:'Минская', prod.columns[8]:'Могилевская'})
    return prod

def loadNotProdPricesRB(path='Average_prices-06-2023.xls', header=7):
    ''' Загрузить непродовольственные товары РБ из файла с ценами
    '''
    df = pd.read_excel(path, header=header)
    notprod = df.iloc[df[df['ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'] == 'НЕПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ'].index[0]+1:]
    notprod = notprod.dropna()
    notprod = notprod.rename(columns={notprod.columns[0]:'Название товара', notprod.columns[1]:'РБ', notprod.columns[2]:'Брестская', notprod.columns[3]:'Витебская', notprod.columns[4]:'Гомельская', notprod.columns[5]:'Гродненская', notprod.columns[6]:'Минск', notprod.columns[7]:'Минская', notprod.columns[8]:'Могилевская'})
    return notprod

def clean_string(string):
  '''Функция очистки
  Получает строку, переводит в нижний регистр, очищает лишних данных, пробелов, знаков препинания и т.д.
  '''
  string = str(string).lower()
  # Ищем и заменяем скобки и все, что в них, на пустую строку
  string = re.sub(r"\(.*\)", " ", string)

  # Удаляем знаки препинания
  string = string.translate(str.maketrans(' ', ' ', s.punctuation))

  # Удаляем единицы измерения
  pattern = r'(\d+)*\s+(кг|г|л|мл|шт|штук|м|м2|м3|см|таблеток|пакетиков)\b' # шаблон регулярного выражения
  string = re.sub(pattern, " ", string) # замена единиц измерения на пустую строку
  # Удаляем цифры
  string = re.sub(r'\d+', " ", string)
  # Удаляем лишние пробелы
  string = re.sub(r"\s+", " ", string)
  
  # Удаляем лишние пробелы в начале и конце строки
  string = string.strip()

  # Добавляем очищенную строку в список
  return string

def clean(df, column):
  '''Применяет функцию clean_string() для всего столбца и записывает новые строки в новый столбец с припиской _cleaned'''
  df[column+'_cleaned'] = df[column].apply(lambda x: clean_string(x))
  return df

prodPricesRB = loadProdPricesRB()
# prodPricesRB = clean(prodPricesRB, prodPricesRB.columns[0])
notProdPricesRB = loadNotProdPricesRB()
# notProdPricesRB = clean(notProdPricesRB, notProdPricesRB.columns[0])

Получение списков товаров РБ с весами

In [268]:
prodRB = prodCategoryRB.GetProducts()
noProdRB = noProdCategoryRB.GetProducts()


Функция сопоставления строк

In [269]:
def my_levenshtein_distance(str1, str2):
   '''Функция для получения разницы между строками методом Левенштейна (не тот мужик из американского пирога),
     при котором строка разбивается на слова и сортируется по алфавиту
   '''
   str1_words = str1.split()
   str2_words = str2.split()

   str1_words.sort()
   str2_words.sort()

   str1_sorted = ' '.join(str1_words)
   str2_sorted = ' '.join(str2_words)

   return levenshtein_distance(str1_sorted, str2_sorted)

def combine_cosine_levenshtein_jar_win(str1, str2, alpha=0.25, beta=0.25, gamma=0.5):
    '''Функция сопоставления двух строк str1 и str2. Возвращает значение от 0 до 1, насколько похожи строки.
    Функция использует комбинацию из методов косинусного сопоставления, Левенштейна и Джаро-Винклера\n
    alpha: вес для метода косинусного сопоставления\n
    beta: вес для метода Левенштейна
    gamma: вес для метода Джаро-Винклера'''
    # convert strings to vectors of character frequencies
    vec1 = list(Counter(str1).values())
    vec2 = list(Counter(str2).values())
    # pad the vectors with zeros to the same length
    vec1, vec2 = pad_sequences([vec1, vec2], padding='post')
    # compute cosine similarity between vectors
    cos_sim = cosine_similarity([vec1], [vec2])[0][0]
    # normalize cosine similarity to [0, 1] range
    cos_sim = (1 + cos_sim) / 2
    # compute levenshtein distance between strings
    lev_dist = my_levenshtein_distance(str1, str2)
    # normalize levenshtein distance to [0, 1] range
    lev_dist = 1 - lev_dist / max(len(str1), len(str2))
    # compute jaro-winkler distance between strings
    jar_win_dist = jaro_winkler_distance.get_jaro_distance(str1, str2)
    # combine the three methods by weighted sum
    return alpha * cos_sim + beta * lev_dist + gamma * jar_win_dist

def GetProductCount(name):
    count_str = re.search(r"(\d+((,+|\.+)*\d*)?\s*((мг)|(г)|(гр)|(кг)|(мл)|(л)))|(1/\d+)", name)
    if count_str == None:
        return 1
    try:
        count_str = count_str.group(0)
        count_str = count_str.replace(",",".")
        if re.search("\s*вес[\s$]+", count_str) != None:
            return 1
        if re.search("1/\d+", count_str) != None:
            count = count_str.replace("1/", "")
            return float(count) / 1000
        if re.search("\d+((,+|\.+)*\d*)?\s*(мг)", count_str) != None:
            count = count_str.replace("мг", "").strip()
            return float(count) / 1000000
        if re.search("\d+((,+|\.+)*\d*)?\s*(кг)", count_str) != None:
            count = count_str.replace("кг", "").strip()
            return float(count)
        if re.search("\d+((,+|\.+)*\d*)?\s*(гр)", count_str) != None:
            count = count_str.replace("гр", "").strip()
            return float(count) / 1000
        if re.search("\d+((,+|\.+)*\d*)?\s*(г)", count_str) != None:
            count = count_str.replace("г", "").strip()
            return float(count) / 1000
        if re.search("\d+((,+|\.+)*\d*)?\s*(мл)", count_str) != None:
            count = count_str.replace("мл", "").strip()
            return float(count) / 1000
        if re.search("\d+((,+|\.+)*\d*)?\s*(л)", count_str) != None:
            count = count_str.replace("л", "").strip()
            return float(count)
    except Exception as e:
        print(e)
        return 1

def compareRB(list1, df2: pd.DataFrame, colname2, perc, noProd=False):
    list2 = df2[colname2].to_list()
    resultdf = pd.DataFrame()
    for i in range(len(list1)):
        name1 = list1[i].name
        if noProd and list1[i].subgroup != None:
            name1 = list1[i].name + ' ' + list1[i].subgroup.name
        name1Clean = clean_string(name1)
        min_dist = 0
        for j in range(len(list2)):
            score = get_cosine_similarity_new(name1Clean, clean_string(list2[j]))
            # score = combine_cosine_levenshtein_jar_win(name1Clean, clean_string(list2[j]), alpha, beta, gamma)
            if score > min_dist:
                min_dist = score
                similar_index = j

        if min_dist >= perc:
            columns = df2.columns.tolist()
            for state in columns[1:]:
                count = GetProductCount(list2[similar_index])
                price = float(df2[state].iloc[similar_index]) / count
                list1[i].SetPriceBYN(state, price)
            resultdf = resultdf.append({'name1': name1,
                                        'name2': list2[similar_index],
                                        'score': min_dist,
                                        'weight': list1[i].weight,
                                        'priceBYN': list1[i].pricesBYN}, ignore_index=True)
        else:
            resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
    return resultdf

def compareRF(list1, df2, colname2, colname2prices, perc, noProd=False):
    list2 = df2[colname2].to_list()
    resultdf = pd.DataFrame()
    for i in range(len(list1)):
        name1 = list1[i].name
        if noProd and list1[i].subgroup != None:
            name1 = list1[i].name + ' ' + list1[i].subgroup.name
        name1Clean = clean_string(name1)
        min_dist = 0
        for j in range(len(list2)):
            # score = match_products(list1[i].name, list2[j], 0, 0, None, None, None, None)
            score = get_cosine_similarity_new(name1Clean, clean_string(list2[j]))
            # score = combine_cosine_levenshtein_jar_win(name1Clean, clean_string(list2[j]), alpha, beta, gamma)
            if score > min_dist:
                min_dist = score
                similar_index = j
        if min_dist >= perc:
            columns = df2.columns.tolist()
            for state in columns[1:]:
                count = GetProductCount(list2[similar_index])
                price = float(df2[colname2prices].iloc[similar_index]) / count
                list1[i].SetPriceRUB(state, price)
            resultdf = resultdf.append({'name1': name1,
                                        'name2': list2[similar_index],
                                        'score': min_dist,
                                        'weight': list1[i].weight,
                                        'priceRUB': list1[i].pricesRUB}, ignore_index=True)
        else:
            resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
    return resultdf

Сопоставление продовольственных товаров РБ

In [271]:
comparedProdRB = compareRB(prodRB, prodPricesRB, prodPricesRB.columns[0],
                        0.75)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Adm

Сопоставление непродовольственных товаров РБ

In [272]:
comparednoProdRB = compareRB(noProdRB, notProdPricesRB, notProdPricesRB.columns[-1], 0.7)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:102: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

Запись в Excel для проверки

In [242]:
comparedProdRB.to_excel('prod_rb2.xlsx')
comparednoProdRB.to_excel('noprod_rb2.xlsx')

Получение списка товаров РФ с ценами

In [273]:
# Загружаем продовольственные и непродовольственные товары РФ
rf = pd.read_excel('! Копия Веса.xlsx', sheet_name='РФ_структура', header=3)
rf = rf.drop(axis=1, columns=[rf.columns[2], rf.columns[3], rf.columns[4], rf.columns[5], rf.columns[6], rf.columns[7], rf.columns[8], rf.columns[9], rf.columns[10]])

Структуризация товаров РФ

In [274]:
prodIndexRF = rf.loc[rf.iloc[:,0] == '01'].index[0]
noProdIndexRF = rf.loc[rf.iloc[:,0] == '03'].index[0]
servicesIndexRF = rf.loc[rf.iloc[:,0] == '04'].index[0]
noProdIndexRF2 = rf.loc[rf.iloc[:, 0] == '05'].index[0]
servicesIndexRF2 = rf.loc[rf.iloc[:,0] == '06'].index[0]

prodCategoryRF = Category('Продовольственные товары')
noProdCategoryRF = Category('Непродовольственные товары')

lastGroup = None
lastSubgroup = None

for i in range(prodIndexRF+1, noProdIndexRF):
    cell = rf.iloc[i, 0]
    name = rf.iloc[i, 1]
    index = cell.split('.')
    if len(index) == 3:
        print('Group: ', name)
        group = Group(name)
        prodCategoryRF.Append(group)
        lastGroup = group
        lastSubgroup = None
    elif len(index) == 4:
        print('Subgroup', name)
        subgroup = Subgroup(name)
        group.Append(subgroup)
        lastSubgroup = subgroup
    elif len(index) == 5 or len(index) == 6:
        print('Product:', name, '-', weight)
        product = Product(name)
        product.SetWeight(weight)
        if lastSubgroup != None:
            product.SetSubgroup(lastSubgroup)
            product.SetGroup(lastGroup)
            lastSubgroup.Append(product)
        else:
            product.SetGroup(lastGroup)
            lastGroup.Append(product)

lastGroup = None
lastSubgroup = None

for i in range(noProdIndexRF+1, servicesIndexRF):
    cell = rf.iloc[i, 0]
    name = rf.iloc[i, 1]
    index = cell.split('.')

    if len(index) == 3:
        print('Group: ',name)
        group = Group(name)
        noProdCategoryRF.Append(group)
        lastGroup = group
        lastSubgroup = None
    elif len(index) == 4:
        print('Subgroup', name)
        subgroup = Subgroup(name)
        group.Append(subgroup)
        lastSubgroup = subgroup
    elif len(index) == 5 or len(index) == 6:
        print('Product:', name, '-', weight)
        product = Product(name)
        product.SetWeight(weight)
        if lastSubgroup != None:
            product.SetSubgroup(lastSubgroup)
            product.SetGroup(lastGroup)
            lastSubgroup.Append(product)
        else:
            product.SetGroup(lastGroup)
            lastGroup.Append(product)

lastGroup = None
lastSubgroup = None

for i in range(noProdIndexRF2+1, servicesIndexRF2):
    cell = rf.iloc[i, 0]
    name = rf.iloc[i, 1]
    index = cell.split('.')

    if len(index) == 3:
        print('Group: ',name)
        group = Group(name)
        noProdCategoryRF.Append(group)
        lastGroup = group
        lastSubgroup = None
    elif len(index) == 4:
        print('Subgroup', name)
        subgroup = Subgroup(name)
        group.Append(subgroup)
        lastSubgroup = subgroup
    elif len(index) == 5 or len(index) == 6:
        print('Product:', name, '-', weight)
        product = Product(name)
        product.SetWeight(weight)
        if lastSubgroup != None:
            product.SetSubgroup(lastSubgroup)
            product.SetGroup(lastGroup)
            lastSubgroup.Append(product)
        else:
            product.SetGroup(lastGroup)
            lastGroup.Append(product)

Group:  Хлебобулочные изделия и крупы (НД)
Product: Рис - 0.0004
Product: Рис шлифованный, кг - 0.0004
Subgroup Мука и другие крупы
Product: Мука - 0.0004
Product: Мука пшеничная, кг - 0.0004
Product: Крупа манная - 0.0004
Product: Крупа манная, кг - 0.0004
Product: Пшено - 0.0004
Product: Пшено, кг - 0.0004
Product: Крупа гречневая - 0.0004
Product: Крупа гречневая-ядрица, кг - 0.0004
Product: Крупы овсяная и перловая - 0.0004
Product: Крупы овсяная и перловая, кг - 0.0004
Subgroup Хлеб
Product: Хлеб пшеничный - 0.0004
Product: Хлеб и булочные изделия из пшеничной муки высшего сорта, кг - 0.0004
Product: Хлеб и булочные изделия из пшеничной муки 1 и 2 сортов, кг - 0.0004
Product: Хлеб ржаной и прочий - 0.0004
Product: Хлеб из ржаной муки и из смеси муки ржаной и пшеничной, кг - 0.0004
Subgroup Другие хлебобулочные изделия
Product: Булочные изделия сдобные - 0.0004
Product: Булочные изделия сдобные из муки высшего сорта штучные, кг - 0.0004
Product: Бараночные изделия - 0.0004
Product:

Функция сопоставления со случайными весами

In [275]:
# Импортируем библиотеку для работы с Excel
import xlsxwriter

def GetDisparity(price1, price2, rate, weight1=1, weight2=1):
    '''
    price1 - цена товара в первой валюте\n
    price2 - цена товара во второй валюте\n
    rate - курс валюты\n
    weight1 - вес товара в первой корзине\n
    weight2 - вес товара во второй корзине
    '''

    return (price1 * weight1 / rate) / (price2 * weight2)


def random_weights():
    '''Возвращает случайные веса для трёх методов
    '''
    alpha = random.random()
    beta = random.random()
    gamma = random.random()
    total = alpha + beta + gamma
    alpha = round(alpha / total, 2)
    beta = round(beta / total, 2)
    gamma = round(gamma / total, 2)
    return alpha, beta, gamma


def compare_rb_rf(list1, list2, perc, rate, alpha = 0.25, beta = 0.5, gamma = 0.25, noProd = False):
    '''Функция нахождения максимально похожих строк, используя цикл и случайные веса
    '''
    # Создаем пустой датафрейм
    resultdf = pd.DataFrame()
    # Получаем список областей из цен РБ и РФ
    states_rb = set()
    states_rf = set()
    for item in list1:
        if item.pricesBYN != None:
            states_rb.update(item.pricesBYN.keys())
    for item in list2:
        if item.pricesRUB != None:
            states_rf.update(item.pricesRUB.keys())
    states_rb = list(states_rb)
    states_rf = list(states_rf)
    # Используем цикл сопоставления
    for i in range(len(list1)):
        min_dist = 0
        max_score = 0
        name1 = clean_string(list1[i].name)
        if noProd and list1[i].subgroup != None:
            name1 = clean_string(list1[i].name + ' ' + list1[i].subgroup.name)
        for j in range(len(list2)):
            name2 = clean_string(list2[j].name)
            
            score = get_cosine_similarity_new(name1, name2)
            # score = compare_all_ngrams_and_return_max_similarity(name1, name2)
            # score = combine_cosine_levenshtein_jar_win(name1, name2, alpha, beta, gamma)
            # score = match_products(list1[i].name, list2[j].name, 0, 0, None, None, None, None)
            if score > min_dist:
                max_score = score
                min_dist = score
                similar_index = j
        # Если нашли похожий товар и у него есть цена в РБ
        if min_dist >= perc and list1[i].pricesBYN != None:
            # Создаем словарь для записи в датафрейм
            print('Товар РБ:', list1[i].name, 'Товар РФ:', list2[similar_index].name)
            row_dict = {'Товар РБ': list1[i].name, 'Доля РБ': list1[i].weight, 'Товар РФ': list2[similar_index].name}
            # Для каждой пары областей считаем диспаритет
            for state_rb in states_rb:
                price_rb = list1[i].pricesBYN.get(state_rb, None)
                row_dict[state_rb] = price_rb
                for state_rf in states_rf:
                    price_rf = list2[similar_index].pricesRUB.get(state_rf, None)
                    if price_rf != None and price_rb != None:
                        list1[i].SetPriceRUB(state_rf, price_rf)
                        row_dict[state_rf] = price_rf
                        # if list1[i].weight != None and list2[similar_index].weight != None:
                        disparity = GetDisparity(price_rb, price_rf, rate) # Стоимость рубля временная. Добавить функцию получения автоматически
                        row_dict['Диспаритет'] = disparity
                        # else:
                        #     row_dict['Диспаритет'] = ''
                    else:
                        row_dict[state_rf] = ''
                        row_dict['Диспаритет'] = ''
            # Записываем в итоговый датафрейм
            resultdf = resultdf.append(row_dict, ignore_index=True)
        # else:
        #     # Если не нашли похожий товар или у него нет цены в РБ, то заполняем пустыми значениями
        #     row_dict = {'Товар РБ': list1[i].name, 'Доля РБ': list1[i].weight, 'Товар РФ': ""}
        #     for state_rb in states_rb:
        #         row_dict[state_rb] = ''
        #         for state_rf in states_rf:
        #             row_dict[state_rf] = ''
        #             row_dict['Диспаритет'] = ''
        #     resultdf = resultdf.append(row_dict, ignore_index=True)
        # Если это не последний товар или если он относится к другой группе товаров, то добавляем строку с названием группы товаров
        if i < len(list1) - 1 and (list1[i+1].group != list1[i].group):
            group_row = {'Товар РБ': list1[i+1].group.name.upper()}
            resultdf = resultdf.append(group_row, ignore_index=True)
    return resultdf


Чтение файла с ценами

In [276]:
rf = pd.read_excel('data.xls', header=2)

rf = rf.rename(columns={'Unnamed: 0': 'name'})
# rf = clean(rf, 'name')

In [277]:
prodRF = prodCategoryRF.GetProducts()
noProdRF = noProdCategoryRF.GetProducts()

Сопоставление продовольственных и непродовольственных товаров РФ-РФ

In [278]:
comparedProdRF = compareRF(prodRF, rf, rf.columns[0], rf.columns[1], 0.7)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users

In [279]:
comparedNoProdRF = compareRF(noProdRF, rf, rf.columns[0], rf.columns[1], 0.7)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': list1[i].name}, ignore_index=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append({'name1': name1,
C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\2923290319.py:127: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.appe

Сопоставление товаров РБ с товарами РФ

In [280]:
prod_rb_rf = compare_rb_rf(prodRB, prodRF, 0.1, 0.035618, 0.35, 0.6, 0.05)

Товар РБ: говядина (кроме бескостного мяса) Товар РФ: Говядина (кроме бескостного мяса), кг


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: говядина бескостная Товар РФ: Говядина бескостная


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: свинина (кроме бескостного мяса) Товар РФ: Свинина (кроме бескостного мяса), кг


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: супы сухие, кубики бульонные  Товар РФ: Сухие супы в пакетах, 100 г


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: жевательная резинка Товар РФ: Жевательная резинка, упаковка


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


In [281]:
noProd_rb_rf = compare_rb_rf(noProdRB, noProdRF, 0.1, 0.035618, 0.27, 0.68, 0.05, noProd=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:99: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(group_row, ignore_index=True)


Товар РБ: куртка утепленная  Товар РФ: Куртка мужская с верхом из плащевых тканей утепленная, шт.


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: пальто (полупальто) демисезонное  Товар РФ: Пальто женское демисезонное из шерстяных или полушерстяных тканей, шт.


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


...

C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: лопата, грабли Товар РФ: Лопата садовая, шт.


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


Товар РБ: спички Товар РФ: Спички, коробок


C:\Users\Admin\AppData\Local\Temp\ipykernel_11920\592679734.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resultdf = resultdf.append(row_dict, ignore_index=True)


In [283]:
prodCategoryRB.to_excel('prod_rb_rf.xlsx', 0.035618)
noProdCategoryRB.to_excel('noProd_rb_rf.xlsx', 0.035618)

In [ ]:
result = pd.concat([prod_rb_rf, noProd_rb_rf])
result.to_excel('result.xlsx')

Запись в Excel для проверки сопоставления

In [ ]:
prod_rb_rf.to_excel('prod_rb_rf.xlsx')
noProd_rb_rf.to_excel('noProd_rb_rf.xlsx')